Both have some very large clusters.  
Better for hierarchial because also have sub clusters.  
    
Note in report that could have tried other clustering methods (kmeans, spectral) but the point was either to get good labels (lingo) or get hierarchy. both have pros over general clustering. kmeans and spectral would get labels in the same way as hierarchy without hierarchy -- one of the struggles is that don't want too many clusters else too much information but then clusters are too big. hierarchy is a way to deal with this

In [34]:
import pickle
import numpy as np
import pandas as pd

### Load Data

In [17]:
with open('lingo', "rb") as f:
    df_final_l = pickle.load(f)
    labels_l = pickle.load(f)
    k_l = pickle.load(f)
    dist_l = pickle.load(f)
    sil_l = pickle.load(f)
    zero = pickle.load(f)

In [16]:
with open('hierarchial', "rb") as f:
    df_final_h = pickle.load(f)
    labels_h = pickle.load(f)
    k_h = pickle.load(f)
    dist_h = pickle.load(f)
    sil_h = pickle.load(f)

In [164]:
hs = set(labels_h.keys())
ls = set(labels_l.keys())
diff = hs.difference(ls) 
diff
# in hierarchial but not lingo because in lingo there were no clusters -- none of the labels fit the documents strongly enough

{'coconut-oil',
 'dmk',
 'income',
 'instal-debt',
 'l-cattle',
 'naphtha',
 'nickel',
 'oat',
 'potato',
 'rape-oil',
 'tea'}

In [165]:
# for comparison, limit hierarchial to lingo searches
df_final_h = df_final_h[df_final_h.search.isin(ls)]
for k in diff:
    labels_h.pop(k, None)
    k_h.pop(k, None)
    dist_h.pop(k, None)
    sil_h.pop(k, None)

### Look for Large Differences in Stats 

In [166]:
def stat_diff(stat1, stat2, n):
    # difference in k: look for big differences
    for i in stat1.keys():
        diff = abs(stat2[i] - stat1[i])
        if diff > n:
            print(i)

In [167]:
stat_diff(k_h, k_l, 5)

alum
lei
heat
soy-oil
platinum
sunseed
lumber
hog


In [218]:
dist_l['sunseed']

0.7100727263292741

In [219]:
dist_h['sunseed']

0.07679295657199192

In [168]:
stat_diff(dist_h, dist_l, 0.5)

lei
heat
soy-oil
groundnut
platinum
retail
sunseed
lumber
hog


In [169]:
stat_diff(sil_l, sil_h, 0.5)

strategic-metal
silver
groundnut
lumber
platinum
rice
meal-feed
heat
lei
sun-oil


### Compare Aggregate Stats

In [170]:
# which has on average higher distortion 
# not surprising lingo is worse -- documents in multiple clusters so sometimes closer to own points 
dist_h_avg = np.mean(list(dist_h.values()))
dist_l_avg = np.mean(list(dist_l.values()))
print('hierarchial:', dist_h_avg)
print('Lingo:', dist_l_avg)

hierarchial: 0.4945076155117177
Lingo: 0.8451787547949818


In [171]:
# which has on average higher silhouette 
# not surprising lingo is worse -- documents in multiple clusters so sometimes closer to own points 
sil_h_avg = np.mean(list(sil_h.values()))
sil_l_avg = np.mean(list(sil_l.values()))
print('hierarchial:', sil_h_avg)
print('Lingo:', sil_l_avg)

hierarchial: 0.4173263105277165
Lingo: 0.13685738750906157


In [172]:
# number of clusters per search term on average
### lingo far fewer...interesting
    # more aggressive about dimensionality reduction 
    # sometimes many documents not in a cluster because there is no good cluster. 
    # doing more post pruning on documents and clusters/labels
    # but then sometimes clusters have wayyyy too many search terms in them because of combining. 
k_l_avg = np.mean(list(k_l.values()))
k_h_avg = np.mean(list(k_h.values()))
print('hierarchial:', k_h_avg)
print('Lingo:', k_l_avg)

hierarchial: 4.968253968253968
Lingo: 3.888888888888889


In [252]:
labels_h['rice']

OrderedDict([(1, ['exports', 'private', 'thailand']),
             (3, ['japan', 'us', 'beef']),
             (2, ['guarantees', 'credit', 'dollars']),
             (4, ['weather', 'winter', 'areas']),
             (5, ['bank', 'indonesias', 'sugar']),
             (7, ['dollars', 'us', 'price']),
             (8, ['exports', 'imports', 'stocks']),
             (6, ['paddy', 'drought', 'crop'])])

In [253]:
labels_l['rice']

OrderedDict([(0, ['exports', 'thailand', 'us']),
             (2, ['credit', 'guarantees', 'dollars']),
             (3, ['price', 'per', 'bank']),
             (1, ['weather', 'areas', 'crop']),
             (4, ['stocks', 'imports', 'production'])])

In [231]:
sil_h['rice']

0.5523855350489011

In [230]:
sil_l['rice']

0.037611398022290694

In [255]:
df_final_h[df_final_h.search == 'sun-oil']

,level_0,index_search,index,categories,ids,text,cluster,search
0,3,0,357,"[grain, groundnut-oil, lin-oil, meal-feed, oil...",test/15500,rpt argentine grainoilseed export prices adju...,3,sun-oil
1,1,1,1757,"[coconut-oil, cotton-oil, palm-oil, rape-oil, ...",test/18781,growth of palm oil use set to slow output to r...,1,sun-oil
2,3,2,3506,"[oilseed, rape-oil, rapeseed, sun-oil, sunseed...",training/10758,algeria sets tender for rapesunflowerseed oil ...,3,sun-oil
3,5,3,6887,"[rape-oil, sun-oil, veg-oil]",training/3950,mexico veg oil tax not aimed at sunflowerusda ...,5,sun-oil
4,2,4,8222,"[corn, grain, lin-oil, oilseed, sorghum, soy-o...",training/6,argentine grainoilseed registrations argent...,2,sun-oil
5,4,5,8465,"[cotton-oil, sun-oil, veg-oil]",training/6412,egypt rejects veg oil offers to retender egy...,4,sun-oil
6,3,6,10271,"[sun-oil, veg-oil]",training/9193,egypt buys of sunflowerseed oil egypt purch...,3,sun-oil


In [251]:
df_final_l[df_final_l.search == 'rice']

,index,categories,ids,text,cluster,len,search
0,3,"[corn, grain, rice, rubber, sugar, tin, trade]",test/14832,thai trade deficit widens in first thailands...,"[4, 0]",2,rice
1,14,"[carcass, corn, grain, livestock, oilseed, ric...",test/14858,japan ministry open farm trade would hit us ...,"[4, 0]",2,rice
2,190,[rice],test/15206,gatt official meets with us farm leaders the...,[0],1,rice
3,282,"[grain, rice]",test/15367,japan ministry has no comment on rice talks re...,[0],1,rice
4,396,"[grain, rice, sorghum, wheat]",test/15567,india foodgrain target in indias national ...,[],0,rice
5,569,"[barley, corn, cotton, cotton-oil, grain, meal...",test/15871,net change in export commitments usda the u...,[],0,rice
6,572,"[barley, corn, cotton, grain, meal-feed, oat, ...",test/15875,us supplydemand detailed by usda the us agri...,"[3, 4]",2,rice
7,596,"[corn, cotton, grain, meal-feed, oilseed, rice...",test/15906,world supplydemand estimates issued by usda ...,"[4, 0]",2,rice
8,614,"[grain, rice]",test/15928,rice supplydemand for thailand usda the us ...,"[3, 0, 4]",3,rice
9,694,"[grain, rice]",test/16079,thai rice exports fall in to thailand expor...,[0],1,rice


In [ ]:
strategic-metal
silver
groundnut
lumber
platinum
rice
meal-feed
heat
lei
sun-oil

### Number of Documents per Cluster per Search Term

Number of documents per cluster for each search term    
For both, grain as one cluster with basically all of the documents. Not ideal. 

In [173]:
# number of documents per cluster for each search term 
grouph = df_final_h.groupby(['search','cluster']).ids.agg('count')
grouph

search   cluster
alum     1            3
         2            5
         3            3
         4            2
         5            4
         6            2
         7            3
         8            6
         9           30
barley   1           14
         2            4
         3            6
         4            4
         5           23
bop      1           25
         2            4
         3           76
carcass  1            8
         2           12
         3           11
         4           37
cocoa    1           21
         2            3
         3           49
coconut  1            5
         2            1
coffee   1           51
         2            4
         3           84
copper   1           13
                   ... 
sunseed  9            1
tin      1            5
         2            9
         3           16
trade    1           19
         2          150
         3          256
         4           30
         5           30
veg-oil  1           33

In [174]:
dfl = df_final_l.copy()[['cluster', 'ids', 'search']]
dfl = dfl[dfl.cluster.str.len() != 0]
dfl = pd.DataFrame([(d, tup.search) for tup in dfl.itertuples() for d in tup.cluster])
dfl.columns = ['cluster', 'search']
dfl['num'] = 1
groupl = dfl.groupby(['search','cluster']).num.agg('count')
groupl

search   cluster
alum     0           41
         1            8
         2            5
barley   0            1
         1           18
         2           17
         3            4
         4            5
bop      0           20
         1           63
         2           66
         3           25
carcass  0           24
         1           25
         2           27
         3           12
         4           26
cocoa    0           34
         1           52
         2           18
         3           10
         4           10
coconut  0            1
         1            2
coffee   0           81
         1           54
         2           22
         3           28
         4           35
         5           29
                   ... 
sunseed  1            8
         2            5
tin      0           15
         1            8
         2           11
trade    0            3
         1          370
         2          158
         3           63
veg-oil  0           72

In [175]:
# number of documents per cluster on average 
print('hierarchial:', grouph.mean())
print('lingo:', groupl.mean())   

hierarchial: 21.881789137380192
lingo: 38.06938775510204


In [259]:
# number of clusters with only one document 
print('hierarchial:',len(grouph[grouph == 1]))
print('lingo:', len(groupl[groupl == 1]))

hierarchial: 37
lingo: 16


In [176]:
labels_h

{'alum': OrderedDict([(1, ['credit', 'azpurua', 'financing']),
              (3, ['alcan', 'close', 'smelter']),
              (6, ['reynolds', 'restart', 'metal']),
              (4, ['british', 'vessel', 'orinoco']),
              (2, ['prices', 'lb', 'cents']),
              (9, ['aluminium', 'plant', 'smelter']),
              (5, ['suralco', 'billiton', 'bauxite']),
              (8, ['lme', 'contract', 'grade']),
              (7, ['ipai', 'ave', 'daily'])]),
 'barley': OrderedDict([(1, ['ecus', 'licences', 'ec']),
              (2, ['bonus', 'ccc', 'accepted']),
              (3, ['wheat', 'uk', 'export']),
              (4, ['reserve', 'v', 'iv']),
              (5, ['crops', 'usda', 'acres'])]),
 'bop': OrderedDict([(3, ['surplus', 'current', 'deficit']),
              (2, ['lire', 'italy', 'showed']),
              (1, ['dollars', 'deficit', 'surplus'])]),
 'carcass': OrderedDict([(1, ['plant', 'union', 'iowa']),
              (2, ['japan', 'beef', 'imports']),
              

In [177]:
labels_h['housing']

OrderedDict([(2, ['units', 'completions', 'fell']),
             (3, ['starts', 'units', 'permits']),
             (1, ['sales', 'homes', 'dollars'])])

In [18]:
labels_l['corn']

OrderedDict([(0, ['us', 'export', 'soviet']),
             (2, ['acres', 'soviet', 'agreement']),
             (3, ['stocks', 'dollars', 'bu']),
             (1, ['inspections', 'thous', 'unknown'])])